In [38]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

CF COMBINED WITH GLOBAL BASELINE APPROACH 

In [39]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990209 entries, 0 to 990208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userID   990209 non-null  int64
 1   movieID  990209 non-null  int64
 2   rating   990209 non-null  int64
dtypes: int64(3)
memory usage: 22.7 MB


In [41]:
Ubase = data.pivot("movieID", "userID", "rating")
Ubase.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
Usample=Ubase.sample(frac=0.25, random_state=42)

In [43]:
train_size = 500
train = Usample[:train_size]
test = Usample[train_size:]

In [ ]:
train.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1150,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
318,4.0,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,4.0,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,3.0,5.0,5.0,NaN,NaN,4.0,5.0,4.0,4.0,4.0,NaN,4.0,NaN,5.0,NaN,NaN,4.0,NaN,5.0,NaN,4.0,NaN,4.0,NaN,5.0,4.0,5.0,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN
2306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1650,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
1934,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [ ]:
#Global Baseline 
# bxi(baseline) = m + bx + bi (m = The average of the movies, bx = Movie bias, bi = User bias)
# rxj = User rating 
# bxj = Score of the movie on global baseline 
# sij = similarity 

1- With Default Metric 

In [ ]:
k=7
myPreds = []
realLabels = []
movietr_list= []
movietest_list =[]
movie_newlist=[]
base_list=[]
mypred=[]

#Calculating movie average in train data
for movieID in train.index:
  movbias_tr = train.dropna(subset=[movieID],axis=1)
  movbias = movbias_tr.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietr_list.append(movbias)
movietr_avg=sum(movietr_list)/len(movietr_list)
print(movietr_avg)

#calculating movie average in test data  
for movieID in test.index:
  movbias_test = test.dropna(subset=[movieID],axis=1)
  movbiasts= movbias_test.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietest_list.append(movbiasts)
movietest_avg=sum(movietest_list)/len(movietest_list)
print(movietest_avg)

for movieID in test.index:
  print(movieID)
  if movieID == 1441:
    break
  for userID in test.columns:
    if not np.isnan(test.loc[movieID, userID]):
      filtered_train = train.dropna(subset=[userID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)

      nn = nei.NearestNeighbors(n_neighbors=k)
      nn.fit(filtered_train)

      if k < len(filtered_train):
        test_row = test.loc[movieID:movieID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0]

        #Calculating baseline in test data 
        userbias_test = test.dropna(subset=[userID])
        userbiasts=userbias_test[userID].mean(axis=0)
        movie_test = test.dropna(subset=[movieID],axis=1)
        moviets= movie_test.loc[movieID:movieID, :].mean(axis=1).values[0]
        base_test=(userbiasts-movietest_avg)+(movbiasts-movietest_avg)+ movietest_avg #global
        #Calculating user bias for train data 
        userbias_train = train.dropna(subset=[userID])
        userbias_tr=userbias_train[userID].mean(axis=0)
        #For train data 
        movieIds = filtered_train.iloc[indices, :].index #movieıds 
        ratings=train.loc[movieIds, userID].values #finding ratings 
        
        movie_newlist=[]
        for id in movieIds:
          movie_trnew = train.dropna(subset=[id],axis=1)
          movienew= movie_trnew.loc[id:id, :].mean(axis=1).values[0]
          movie_newlist.append(movienew)
        
        base_list=[]
        for i in range(len(movie_newlist)):
          base_train=(userbias_tr-movietr_avg)+(movie_newlist[i]-movietr_avg)+movietr_avg
          base_list.append(base_train)  
        
        mypred=[]
        for i in range(len(base_list)):
          pred=ratings[i]- base_list[i]  #Formülün pay kısmı 
          mypred.append(pred)
        
        result =np.sum(mypred*distances)/np.sum(distances) #Formülün payda kısmı 
        pred=base_test+result #Formülün tamamı 
        
        myPreds.append(pred)
        
        real_label = test.loc[movieID, userID]
        realLabels.append(real_label)

3.282686505962011
3.1974528031389005
318
2306
1650
1934
1279
446
2409
1441


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.1395085054472978

2 - With Correlation Metric 

In [44]:
k=7
myPreds = []
realLabels = []
movietr_list= []
movietest_list =[]
movie_newlist=[]
base_list=[]
mypred=[]

#Calculating movie average in train data
for movieID in train.index:
  movbias_tr = train.dropna(subset=[movieID],axis=1)
  movbias = movbias_tr.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietr_list.append(movbias)
movietr_avg=sum(movietr_list)/len(movietr_list)
print(movietr_avg)

#calculating movie average in test data  
for movieID in test.index:
  movbias_test = test.dropna(subset=[movieID],axis=1)
  movbiasts= movbias_test.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietest_list.append(movbiasts)
movietest_avg=sum(movietest_list)/len(movietest_list)
print(movietest_avg)

for movieID in test.index:
  print(movieID)
  if movieID == 1441:
    break
  for userID in test.columns:
    if not np.isnan(test.loc[movieID, userID]):
      filtered_train = train.dropna(subset=[userID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)

      nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
      nn.fit(filtered_train)

      if k < len(filtered_train):
        test_row = test.loc[movieID:movieID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0]

        #Calculating baseline in test data 
        userbias_test = test.dropna(subset=[userID])
        userbiasts=userbias_test[userID].mean(axis=0)
        movie_test = test.dropna(subset=[movieID],axis=1)
        moviets= movie_test.loc[movieID:movieID, :].mean(axis=1).values[0]
        base_test=(userbiasts-movietest_avg)+(movbiasts-movietest_avg)+ movietest_avg #global
        #Calculating user bias for train data 
        userbias_train = train.dropna(subset=[userID])
        userbias_tr=userbias_train[userID].mean(axis=0)
        #For train data 
        movieIds = filtered_train.iloc[indices, :].index #movieıds 
        ratings=train.loc[movieIds, userID].values #finding ratings 
        
        movie_newlist=[]
        for id in movieIds:
          movie_trnew = train.dropna(subset=[id],axis=1)
          movienew= movie_trnew.loc[id:id, :].mean(axis=1).values[0]
          movie_newlist.append(movienew)
        
        base_list=[]
        for i in range(len(movie_newlist)):
          base_train=(userbias_tr-movietr_avg)+(movie_newlist[i]-movietr_avg)+movietr_avg
          base_list.append(base_train)  
        
        mypred=[]
        for i in range(len(base_list)):
          pred=ratings[i]- base_list[i]  #Formülün pay kısmı 
          mypred.append(pred)
        
        result =np.sum(mypred*distances)/np.sum(distances) #Formülün payda kısmı 
        pred=base_test+result #Formülün tamamı 
        
        myPreds.append(pred)
        
        real_label = test.loc[movieID, userID]
        realLabels.append(real_label)

3.282686505962011
3.1974528031389005
318
2306
1650
1934
1279
446
2409
1441


In [45]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.0637254225493884

In [ ]:
k=10
myPreds = []
realLabels = []
movietr_list= []
movietest_list =[]
movie_newlist=[]
base_list=[]
mypred=[]

#Calculating movie average in train data
for movieID in train.index:
  movbias_tr = train.dropna(subset=[movieID],axis=1)
  movbias = movbias_tr.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietr_list.append(movbias)
movietr_avg=sum(movietr_list)/len(movietr_list)
print(movietr_avg)

#calculating movie average in test data  
for movieID in test.index:
  movbias_test = test.dropna(subset=[movieID],axis=1)
  movbiasts= movbias_test.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietest_list.append(movbiasts)
movietest_avg=sum(movietest_list)/len(movietest_list)
print(movietest_avg)

for movieID in test.index:
  print(movieID)
  if movieID == 1441:
    break
  for userID in test.columns:
    if not np.isnan(test.loc[movieID, userID]):
      filtered_train = train.dropna(subset=[userID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)

      nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
      nn.fit(filtered_train)

      if k < len(filtered_train):
        test_row = test.loc[movieID:movieID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0]
        #Calculating baseline in test data 
        userbias_test = test.dropna(subset=[userID])
        userbiasts=userbias_test[userID].mean(axis=0)
        movie_test = test.dropna(subset=[movieID],axis=1)
        moviets= movie_test.loc[movieID:movieID, :].mean(axis=1).values[0]
        base_test=(userbiasts-movietest_avg)+(movbiasts-movietest_avg)+ movietest_avg #global
        #Calculating user bias for train data 
        userbias_train = train.dropna(subset=[userID])
        userbias_tr=userbias_train[userID].mean(axis=0)
        #For train data 
        movieIds = filtered_train.iloc[indices, :].index #movieıds 
        ratings=train.loc[movieIds, userID].values #finding ratings 
        
        movie_newlist=[]
        for id in movieIds:
          movie_trnew = train.dropna(subset=[id],axis=1)
          movienew= movie_trnew.loc[id:id, :].mean(axis=1).values[0]
          movie_newlist.append(movienew)
        
        base_list=[]
        for i in range(len(movie_newlist)):
          base_train=(userbias_tr-movietr_avg)+(movie_newlist[i]-movietr_avg)+movietr_avg
          base_list.append(base_train)  
        
        mypred=[]
        for i in range(len(base_list)):
          pred=ratings[i]- base_list[i]  #Formülün pay kısmı 
          mypred.append(pred)
        
        result =np.sum(mypred*distances)/np.sum(distances) #Formülün payda kısmı 
        pred=base_test+result #Formülün tamamı 
        
        myPreds.append(pred)
        
        real_label = test.loc[movieID, userID]
        realLabels.append(real_label)

3.282686505962011
3.1974528031389005
318
2306
1650
1934
1279
446
2409
1441


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.0523434263373366

In [ ]:
k=12
myPreds = []
realLabels = []
movietr_list= []
movietest_list =[]
movie_newlist=[]
base_list=[]
mypred=[]

#Calculating movie average in train data
for movieID in train.index:
  movbias_tr = train.dropna(subset=[movieID],axis=1)
  movbias = movbias_tr.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietr_list.append(movbias)
movietr_avg=sum(movietr_list)/len(movietr_list)
print(movietr_avg)

#calculating movie average in test data  
for movieID in test.index:
  movbias_test = test.dropna(subset=[movieID],axis=1)
  movbiasts= movbias_test.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietest_list.append(movbiasts)
movietest_avg=sum(movietest_list)/len(movietest_list)
print(movietest_avg)

for movieID in test.index:
  print(movieID)
  if movieID == 1441:
    break
  for userID in test.columns:
    if not np.isnan(test.loc[movieID, userID]):
      filtered_train = train.dropna(subset=[userID])
      filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
      filtered_train.fillna(0, inplace=True)

      nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
      nn.fit(filtered_train)

      if k < len(filtered_train):
        test_row = test.loc[movieID:movieID, :]
        test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
        test_row.fillna(0, inplace=True)

        indices = nn.kneighbors(test_row, return_distance=False)[0]
        distances=nn.kneighbors(test_row)[0]
        #Calculating baseline in test data 
        userbias_test = test.dropna(subset=[userID])
        userbiasts=userbias_test[userID].mean(axis=0)
        movie_test = test.dropna(subset=[movieID],axis=1)
        moviets= movie_test.loc[movieID:movieID, :].mean(axis=1).values[0]
        base_test=(userbiasts-movietest_avg)+(movbiasts-movietest_avg)+ movietest_avg #global
        #Calculating user bias for train data 
        userbias_train = train.dropna(subset=[userID])
        userbias_tr=userbias_train[userID].mean(axis=0)
        #For train data 
        movieIds = filtered_train.iloc[indices, :].index #movieıds 
        ratings=train.loc[movieIds, userID].values #finding ratings 
        
        movie_newlist=[]
        for id in movieIds:
          movie_trnew = train.dropna(subset=[id],axis=1)
          movienew= movie_trnew.loc[id:id, :].mean(axis=1).values[0]
          movie_newlist.append(movienew)
        
        base_list=[]
        for i in range(len(movie_newlist)):
          base_train=(userbias_tr-movietr_avg)+(movie_newlist[i]-movietr_avg)+movietr_avg
          base_list.append(base_train)  
        
        mypred=[]
        for i in range(len(base_list)):
          pred=ratings[i]- base_list[i]  #Formülün pay kısmı 
          mypred.append(pred)
        
        result =np.sum(mypred*distances)/np.sum(distances) #Formülün payda kısmı 
        pred=base_test+result #Formülün tamamı 
        
        myPreds.append(pred)
        
        real_label = test.loc[movieID, userID]
        realLabels.append(real_label)

3.282686505962011
3.2255122148487834
318
2306
1650
1934
1279
446
2409
1441


In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

1.4059299794499531

Prediction with Global Baseline Approach

In [ ]:
test = pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [ ]:
x = pd.DataFrame(test.IDs.str.split('_',1).tolist(),
                         columns = ['userId','movieId'])
print(x)

     userId movieId
0       629    2683
1       601     904
2      5673    3717
3      5616    1721
4      1099    3697
...     ...     ...
9994   4198    3671
9995    566    3148
9996   5531    2105
9997   4650     377
9998   4919    2094

[9999 rows x 2 columns]


In [ ]:
df_test = pd.DataFrame(x)
df_test

,userId,movieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697
...,...,...
9994,4198,3671
9995,566,3148
9996,5531,2105
9997,4650,377


In [ ]:
df_test['userId'] = df_test['userId'].astype(str).astype(int)
df_test['movieId'] = df_test['movieId'].astype(str).astype(int)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   userId   9999 non-null   int64
 1   movieId  9999 non-null   int64
dtypes: int64(2)
memory usage: 156.4 KB


In [ ]:
movietest_list=[]
for movieID in Ubase.index:
  movbias_test = Ubase.dropna(subset=[movieID],axis=1)
  movbiasts= movbias_test.loc[movieID:movieID, :].mean(axis=1).values[0]
  movietest_list.append(movbiasts)
movietest_avg=sum(movietest_list)/len(movietest_list)

k = 10
myPreds = []
for i in range(len(df_test.index)):
  userId = df_test.loc[i].at["userId"]
  movieId = df_test.loc[i].at["movieId"]
  filtered_train = Ubase.dropna(subset=[userId])
  filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
  filtered_train.fillna(0, inplace=True)

  nn = nei.NearestNeighbors(n_neighbors=k,metric='correlation')
  nn.fit(filtered_train)

  if k < len(filtered_train):
    test_row = Ubase.loc[movieID:movieID, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]
    distances=nn.kneighbors(test_row)[0]
    userbias = Ubase.dropna(subset=[userId])
    userbias= userbias[userId].mean(axis=0)
    moviebias = Ubase.dropna(subset=[movieID],axis=1)
    moviebias= moviebias.loc[movieID:movieID, :].mean(axis=1).values[0]
    base_test=movietest_avg+ (userbias-movietest_avg)+(moviebias-movietest_avg) #global

    movieIds = filtered_train.iloc[indices, :].index #movieıds 
    ratings=Ubase.loc[movieIds, userId].values #finding ratings 
        
    movie_newlist=[]
    for id in movieIds:
      movie_trnew = Ubase.dropna(subset=[id],axis=1)
      movie_trnew= movie_trnew.loc[id:id, :].mean(axis=1).values[0]
      movie_newlist.append(movie_trnew)
        
    base_list=[]
    for i in range(len(movie_newlist)):
      base_new=movietest_avg+ (userbias-movietest_avg)+(movie_newlist[i]-movietest_avg)
      base_list.append(base_new)  
        
    mypredlist=[]
    for i in range(len(base_list)):
      pred2=ratings[i]- base_list[i]  #Formülün pay kısmı 
      mypredlist.append(pred2)
        
    result =np.sum(mypredlist*distances)/np.sum(distances) #Formülün payda kısmı 
    pred=base_test+result #Formülün tamamı 
        
    myPreds.append(pred)
    

In [ ]:
len(myPreds)

9999

In [ ]:
test2 = test.copy()
test2["rating"]= myPreds

In [ ]:
test2.head()

,IDs,rating
0,629_2683,3.841358
1,601_904,4.451311
2,5673_3717,4.214161
3,5616_1721,3.611798
4,1099_3697,4.115159


In [ ]:
from google.colab import files
test2.to_csv('submission10.csv',index=False) 
files.download('submission10.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>